In [17]:
# Import the class libraries 
import requests
import ssl
import json
from pprint import pprint
from requests import Response
import pandas as pd
import time
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from IPython.display import IFrame
from IPython.display import display, HTML
from pandas import json_normalize
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

First we create a helper class that encapsulates the methods of the **Data Virtualization REST API**.

In [18]:
class Helper():
    
    def __init__(self, url, verify = False, proxies=None, ):
        self.url = url
        self.proxies = proxies
        self.verify = verify

    def printResponse(self, r, code):
        if (r.status_code == code):
            pprint(r.json())
        else:
            print (r.status_code)
            print (r.content)
            
    def getRequest(self, api, json=None):
        return requests.get(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json)

    def postRequest(self, api, json=None):
        return requests.post(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
    
    def deleteRequest(self, api, json=None):
        return requests.delete(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
        
    def getStatusCode(self, response):
        return (response.status_code)

    def getJSON(self, response):
        return (response.json())
    
    # ----------------------------------------------------------------------------------------
    # The next section defines the methods of the Data Virtualization REST API
    
    def authenticate(self, api, userid, password):
        
        credentials = {'username':userid, 'password':password}
        r = requests.post(self.url+api+'/preauth/signin', verify=self.verify, json=credentials, proxies=self.proxies)
        if (r.status_code == 200):
            bearerToken = "Bearer " + r.cookies["ibm-private-cloud-session"]
            print('Token Retrieved')
            self.headers = {'Content-Type':"application/json", 'Accept':"application/json", 'Authorization': bearerToken, 'Cache-Control': "no-cache"}
        else:
            print ('Unable to authenticate, no bearer token obtained')
    
    def getSchemas(self):
        return self.getRequest('/icp4data-databases/dv/cp4d/dbapi/v4/schemas')
           
    def getSearchViewList(self, searchtext, show_systems="false"):
        return self.getRequest('/icp4data-databases/dv/cp4d/dbapi/v4/admin/schemas/obj_type/view?search_name='+searchtext+'&show_systems='+str(show_systems)+'&rows_return=200');
    
    def getSearchTableList(self, searchtext):
        return self.getRequest('/icp4data-databases/dv/cp4d/dbapi/v4/admin/schemas/obj_type/table?search_name='+searchtext+'&show_systems=true&rows_return=100');
               
    def postSearchObjects(self, obj_type, search_text, rows_return=100, show_systems='false', is_ascend='true'):     
        json = {"search_name":search_text,"rows_return":rows_return,"show_systems":show_systems,"obj_type":obj_type,"filters_match":"ALL","filters":[]}       
        return self.postRequest('/icp4data-databases/dv/cp4d/dbapi/v4/admin/'+str(obj_type)+'s',json);
            
    def getTablesInSchema(self, schema):
        return self.getRequest('/icp4data-databases/dv/cp4d/dbapi/v4/schemas/'+str(schema)+'/tables'); 
    
    def getVirtualizedTables(self):
        return self.getRequest('/icp4data-databases/dv/cp4d/dvapiserver/v1/dv/mydata/tables')
    
    def getVirtualizedTablesDF(self):
        r = self.getVirtualizedTables()
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r)
            df = pd.DataFrame(json_normalize(json['tables']))
            return df
        else:
            print(self.getStatusCode(r))

    def getVirtualizedViews(self):
        return self.getRequest('/icp4data-databases/dv/cp4d/dvapiserver/v1/dv/mydata/views')
    
    def getVirtualizedViewsDF(self):
        r = self.getVirtualizedViews()
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r)
            df = pd.DataFrame(json_normalize(json['views']))
            return df
        else:
            print(self.getStatusCode(r))
    
    def grantPrivledgeToRole(self, objectName, objectSchema, roleToGrant):
        json =   {"objectName":objectName,"objectSchema":objectSchema,"roleToGrant":roleToGrant}
        return self.postRequest('/icp4data-databases/dv/cp4d/dvapiserver/v1/privileges/roles',json);
 
    def getRole(self, role):
        return self.getRequest('/icp4data-databases/dv/cp4d/dvapiserver/v1/privileges/objects/role/'+str(role));
    
    def foldData(self, sourceName, sourceTableDef, sources ):
        json = {"sourceName":sourceName,"sourceTableDef":sourceTableDef,"sources":sources}
        return self.postRequest('/icp4data-databases/dv/cp4d/dvapiserver/v1/dv/virtualize/tables', json);

    def addUser(self, username, displayName, email, user_roles, password):
        json = {"username":username,"displayName":displayName,"email":email,"user_roles":user_roles,"password":password}
        return self.postRequest('/api/v1/usermgmt/v1/user', json);
    
    def dropUser(self, username):
        return self.deleteRequest('/api/v1/usermgmt/v1/user/'+str(username));
   
    def getUsers(self):
        return self.getRequest('/api/v1/usermgmt/v1/usermgmt/users');
    
    def getUsersDF(self):
        r = self.getUsers()
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r)
            df = pd.DataFrame(json_normalize(json))
            return df
        else:
            print(self.getStatusCode(r));
    
    def addUserToDV(self, display_name, role, usersDF):
        userrow = (usersDF.loc[usersDF['displayName'] == display_name])
        uid = userrow['uid'].values[0]
        username = userrow['username'].values[0]
        
        json = {"users":[{"uid":uid,"username":username,"display_name":display_name,"role":role}],"serviceInstanceID":"1610031237189593"}
        return self.postRequest('/zen-data/v2/serviceInstance/users', json);
    
    def dropUserFromDV(self, display_name, usersDF):
        userrow = (usersDF.loc[usersDF['displayName'] == display_name])
        uid = userrow['uid'].values[0]
        
        json = {"users":[uid],"serviceInstanceID":"1610031237189593"}
        return self.deleteRequest('/zen-data/v2/serviceInstance/users', json);
    
    def deleteVirtualizedTable(self, table_schema, table_name, data_source_table_name):
        payload = {"table_schema":table_schema,"table_name":table_name,"data_source_table_name":data_source_table_name}
        return self.deleteRequest('/icp4data-databases/dv/cp4d/dbapi/v4/dv/federation', payload);
    
    def deleteView(self, schema, view):
        return self.deleteRequest('/icp4data-databases/dv/cp4d/dbapi/v4/dv/federation/views/'+str(schema)+'/'+str(view))

    def getDataSourcesAPI(self):
        return self.getRequest('/icp4data-databases/dv/cp4d/dvapiserver/v1/dv/datasource_nodes')
    
    def getDataSources(self):
        columns = ['cid','connection_id', 'dbname', 'srchostname', 'srcport','srctype','status','usr','uri']
        dfTotal = pd.DataFrame(columns=columns)
        r = self.getDataSourcesAPI()
        if (self.getStatusCode(r)==200):
            json = self.getJSON(r)
            df = pd.DataFrame(json_normalize(json))
            for index, row in df.iterrows():
                if row['agent_class']!='F':
                    dfTotal = pd.concat([dfTotal, pd.DataFrame(json_normalize(row['dataSources']))],ignore_index=True)
            return(dfTotal[['srctype','srchostname', 'srcport', 'dbname', 'usr', 'status']])
        else:
            print(getStatusCode(r))   

# Authenticate at the Data Virtualization REST server
To access the REST API methods we first need to first authenticate and retrieve a token. 
The token can be reused for all following calls to the API methods. 
This ensures that we don't have to provide a userID and password each time we execute a method. 

In [19]:
# Set the Data Virtualization REST API endpoint
dv_endpoint  = 'https://10.1.1.1:30633'

# Initialize the helper class
CPDAPI = Helper(dv_endpoint)

# Authenticate at the Data Virtualization REST server
api = '/v1'
user = 'admin'
password = 'password'
CPDAPI.authenticate(api, user, password)

# database = Console

Token Retrieved


### Display all DV data sources

In [20]:
display(CPDAPI.getDataSources())

,srctype,srchostname,srcport,dbname,usr,status
0,DB2,10.1.1.1,31895,bludb,admin,Up
1,MongoDB,10.1.1.12,27017,mongo_onprem_stocks,admin,Up
2,DB2,35.172.245.247,32403,bludb,db2inst1,Up
3,PostgreSQL,10.1.1.1,32688,postgres,admin,Up
4,MongoDB,server7,27017,mongo_onprem_stocks,admin,Up
5,Informix,server7,26111,STOCKS,informix,Up
6,DB2,server7,50000,ONTIME15,db2inst1,Up
7,DB2,server7,50000,ONTIME13,db2inst1,Up
8,DB2,server7,50000,ONTIME14,db2inst1,Up
9,DB2,server7,50000,ONTIME11,db2inst1,Up


### Display all Virtualized Views

In [22]:
display(CPDAPI.getVirtualizedViewsDF())

,create_time,owner,viewname,viewschema
0,1632232426628,ADMIN,STOCK_TRANSACTIONS,TRADING
1,1611091127163,ADMIN,TOPBOUGHTSOLD,TRADING
2,1611092282515,ADMIN,OHIO,TRADING
3,1632232426910,ADMIN,ACCOUNTS,TRADING
4,1611181656237,ADMIN,CUSTOMERS_CONTACT,MONGO
5,1611181716826,ADMIN,CUSTOMERS_IDENTITY,MONGO
6,1632851761853,ADMIN,AIRCRAFT,ONTIME
7,1611085760420,ADMIN,CUSTOMER_MONGO,TRADING
8,1611090761096,ADMIN,TOPBYSELL,TRADING
9,1611181763944,ADMIN,CUSTOMERS_PAYMENT,MONGO


### Display all Data Virtualization users

In [23]:
display(CPDAPI.getUsersDF())

,uid,username,displayName,email,approval_status,permissions,user_roles,current_account_status,internal_user,deletable,authenticator,created_timestamp,last_modified_timestamp,role
0,1000330999,admin,admin,--,approved,"[author_governance_artifacts, manage_governanc...","[zen_administrator_role, zen_developer_role, w...",enabled,False,False,default,1606426492292,1606438026273,Admin
1,1000331001,steward,Data Steward,stewkohlmann@ca.ibm.com,approved,"[author_governance_artifacts, view_governance_...","[wkc_data_steward_role, zen_administrator_role...",enabled,False,True,default,1610135029441,1610135232741,Admin
2,1000331002,dataengineer,DATAENGINEER,testdataengineer@ca.ibm.com,approved,"[author_governance_artifacts, view_governance_...",[zen_data_engineer_role],enabled,False,True,default,1610488584358,1610488584358,User
3,1000331003,labuser,LABUSER,xxxx@ca.ibm.com,approved,[access_catalog],[wkc_data_scientist_role],enabled,False,True,default,1611177435451,1636135848400,User
4,1000331004,dataengineer0,DATAENGINEER0,xxxx@ca.ibm.com,approved,"[author_governance_artifacts, view_governance_...",[zen_data_engineer_role],enabled,False,True,default,1611177878811,1611177878811,User
5,1000331005,dataengineer1,DATAENGINEER1,xxxx@ca.ibm.com,approved,"[author_governance_artifacts, view_governance_...",[zen_data_engineer_role],enabled,False,True,default,1611177879659,1611177879659,User
6,1000331006,dataengineer2,DATAENGINEER2,xxxx@ca.ibm.com,approved,"[author_governance_artifacts, view_governance_...",[zen_data_engineer_role],enabled,False,True,default,1611177880628,1611177880628,User
7,1000331007,dataengineer3,DATAENGINEER3,xxxx@ca.ibm.com,approved,"[author_governance_artifacts, view_governance_...",[zen_data_engineer_role],enabled,False,True,default,1611177881434,1611177881434,User
8,1000331008,dataengineer4,DATAENGINEER4,xxxx@ca.ibm.com,approved,"[author_governance_artifacts, view_governance_...",[zen_data_engineer_role],enabled,False,True,default,1611177882226,1611177882226,User
9,1000331009,dataengineer5,DATAENGINEER5,xxxx@ca.ibm.com,approved,"[author_governance_artifacts, view_governance_...",[zen_data_engineer_role],enabled,False,True,default,1611177883213,1611177883213,User
